In [53]:
## imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans,DBSCAN
import nltk
import re

In [45]:
### dataframes
df = pd.read_csv("Raw_Skills_Dataset.csv")
sample = pd.read_csv("Example_Technical_Skills.csv")

In [46]:
print("Size of data : ",df.shape[0])
df.sample(5)

Size of data :  34116


,RAW DATA
5015,various network protocols
16903,the functional/ performance/ user experience r...
30072,Veracode
22993,Active Directory
5070,data analytics



Things to preprocess: 

1. Some of the points contains . which is not useful removing them.
2. SOme have special character.
3. Lot of dublicates. 
4. Remove charcter such as .


WorkFlow:

1. Lower the words.
2. Check dublicates.
3. Remove punctuation.
4. Check dublicates.


In [47]:
### remove duplicatese
df_without_duplicates = df.drop_duplicates(keep="first")
print("Size after duplicate remove : ",df_without_duplicates.shape[0],":",\
      ((df.shape[0]-df_without_duplicates.shape[0])/df.shape[0])*100)

Size after duplicate remove :  15677 : 54.04795403916051


In [48]:
## lower text
lower = df_without_duplicates["RAW DATA"].apply(lambda x: x.lower())

In [49]:
df_without_duplicates["RAW DATA"] = lower

C:\Users\dell\AppData\Local\Temp/ipykernel_17012/61770665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_duplicates["RAW DATA"] = lower


In [50]:
df_without_duplicates

,RAW DATA
0,what ifs
1,seniority
2,familiarity
3,functionalities
4,lambdas
...,...
34106,leadership qualities
34108,actalent
34109,all applicants
34110,"self-motivated, enthusiastic and strong drive"


In [51]:
### remove duplicatese
df_without_duplicates = df_without_duplicates.drop_duplicates(keep="first")


In [52]:
df_without_duplicates.shape[0]

14892

In [57]:
### removing special character
df_without_duplicates["RAW DATA"] = df_without_duplicates["RAW DATA"].apply(lambda x: re.sub('[^A-Za-z0-9]+'," ",x))

C:\Users\dell\AppData\Local\Temp/ipykernel_17012/3854185715.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_duplicates["RAW DATA"] = df_without_duplicates["RAW DATA"].apply(lambda x: re.sub('[^A-Za-z0-9]+'," ",x))


In [60]:
df_without_duplicates = df_without_duplicates.drop_duplicates(keep="first")

In [70]:
df_without_duplicates.reset_index().drop("index",axis=1)

,RAW DATA
0,what ifs
1,seniority
2,familiarity
3,functionalities
4,lambdas
...,...
14691,leadership qualities
14692,actalent
14693,all applicants
14694,self motivated enthusiastic and strong drive


### Vectorizer

1. TFIDF

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = df_without_duplicates["RAW DATA"].values
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(corpus)

In [92]:
k_means = KMeans(n_clusters=2)
k_means.fit(x_train)


KMeans(n_clusters=2)

In [95]:
cluster = k_means.transform(x_train)

In [96]:
cluster_df = pd.DataFrame(cluster)

In [105]:
##concat
df_without_duplicates.reset_index(inplace=True)
dataframe = pd.concat([df_without_duplicates,cluster_df],axis=1)

In [107]:
dataframe["Labels"] = k_means.labels_

In [176]:
dataframe[dataframe.Labels==0].sample(10)  ## non techniqual 

,level_0,index,RAW DATA,0,1,Labels
12717,12717,29066,good scripting,0.998482,1.021530,0
13791,13791,31454,ubuntu,1.001170,1.032267,0
7339,7339,17480,uscareer synechron com,1.001050,1.032267,0
11959,11959,27501,reusable software,0.995193,1.023078,0
4382,4382,11660,a timely fashion,1.000643,1.028193,0
3269,3269,9462,rdbms experience,0.994544,1.026199,0
4022,4022,11036,qat,1.001303,1.032267,0
14150,14150,32650,jms mq,1.000984,1.032094,0
6188,6188,15321,release orchestration tool,0.999757,1.031165,0
7893,7893,18515,the status quo,0.996186,1.030073,0


In [182]:
dataframe[dataframe.Labels==1].sample(10)    ## techniqual skills 

,level_0,index,RAW DATA,0,1,Labels
12994,12994,29590,groundbreaking and multifaceted technologies,0.999610,0.980861,1
987,987,1520,clean architectural and coding principles,0.998565,0.971068,1
11269,11269,24911,devops and code versioning tools,0.994963,0.966356,1
11544,11544,25466,excellent verbal and written english communica...,1.000261,0.914868,1
3921,3921,10776,data lake and digital platform capabilities,0.996118,0.977081,1
8537,8537,19781,desktop and mobile applications,0.998033,0.965547,1
5604,5604,14345,embedded and structural composites,1.000286,0.983568,1
11458,11458,25274,both business and technical specialists,0.997441,0.963891,1
6360,6360,15593,useful manual and automated software testing e...,0.994718,0.956243,1
11514,11514,25383,logical analytical and debugging problem solvi...,0.999240,0.925292,1


#### Using PCA and kmeans

In [183]:
from sklearn.decomposition import PCA

In [265]:
parameter = [5,10,15,50,100,150,200]
pca = PCA(n_components=500)
values = pca.fit_transform(x_train.A)


In [266]:
k_means_pca = KMeans(n_clusters=2)
k_means_pca.fit(values)


KMeans(n_clusters=2)

In [267]:
k_means_pca.labels_

array([0, 0, 0, ..., 0, 0, 0])

In [268]:
# df_without_duplicates.drop(["level_0","index"],axis=1,inplace=True)

In [269]:
df_without_duplicates["labels"] = k_means_pca.labels_

In [279]:
df_without_duplicates[df_without_duplicates.labels==1].sample(10)   ## contains techniqual skills

,RAW DATA,labels
8207,the mongo database,0
5843,the dsp team,0
7915,the aws cloud architecture solution,0
13524,the lifecycle planning,0
5126,the pulse secure desktop client,0
2375,the necessary security mitigations,0
1998,the sas programming group,0
1669,the automated build process,0
14004,the thingspace iot platform,0
13085,the operational characteristics,0


In [316]:
df_without_duplicates[df_without_duplicates.labels==0].sample(10)   ### contains non techniqual skills 

,RAW DATA,labels
6664,carrier grade telecommunications products,0
7790,database fundamentals,0
11520,java scala,0
4279,servicewatch,0
4452,our software development lifecycle,0
13169,any patents,0
6865,analysts,0
1073,scalable and efficient data processing systems,1
9564,git knowledge,0
6964,architecting,0
